In [41]:
# 표준 오토인코더
# 확장을 해서 변이형 오토인코더

In [42]:
import tensorflow as tf
from tensorflow.keras import datasets
(x_train,y_train),(x_test,y_test) = datasets.fashion_mnist.load_data()

In [43]:
x_train.shape

(60000, 28, 28)

In [44]:
import numpy as np

In [45]:
# 첫번째 차원은 가장 바깥쪽 차원을 0개 추가 그래서 변하지 않음
# 두번째 차원은 좌,우 각각 2개의 값으로 추가, 좌우 각각2열이 0으로 패딩이 추가
# 세번째 차원은 위,아래 각각 2개의 값으로 추가, 좌우 각각2열이 0으로 패딩이 추가
temp = np.pad(x_train,((0,0),(2,2),(2,2)),constant_values=0.0)
print(temp.shape)
np.expand_dims(temp,-1).shape

(60000, 32, 32)


(60000, 32, 32, 1)

In [46]:
# 전처리 : float32, /255.0
# 각 이미지에 패딩을 적용해서 32 x 32 : 신경망 통과시 텐서의 크기를 쉽게 조작할수 있도록
def preprocess(imgs):
  imgs = imgs.astype('float32') / 255.0
  imgs = np.pad(imgs,((0,0),(2,2),(2,2)),constant_values=0.0)
  imgs = np.expand_dims(imgs,-1)
  return imgs
x_train = preprocess(x_train)
x_test = preprocess(x_test)

In [47]:
# 인코더 : 고차원 입력데이터를 저차원 임베딩 벡터로 압축
# 디코더 : 임베딩 벡터를 원본 도메인으로 압축 해제(이미지로 되돌린다)
# 인코더를 수행하면...특징들을 모은 임베딩(잠재공간).. 임베딩을 샘플링해서 디코더에 넣으면 새로운 이미지생성

In [48]:
from tensorflow.keras import layers,models
import tensorflow.keras.backend as k

In [49]:
k.int_shape(x_train)

(60000, 32, 32, 1)

In [50]:
#인코더
encorder_input = layers.Input(shape=(32, 32, 1))
x = layers.Conv2D(32,(3,3),strides=2,activation='relu',padding='same')(encorder_input)
x = layers.Conv2D(64,(3,3),strides=2,activation='relu',padding='same')(x)
x = layers.Conv2D(128,(3,3),strides=2,activation='relu',padding='same')(x)
shape_before_flatten = k.int_shape(x)[1:]  # 디코더에서 사용
x = layers.Flatten()(x)
encorder_output = layers.Dense(2)(x)
encorder = models.Model(encorder_input,encorder_output)

In [51]:
encorder.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_21 (Conv2D)          (None, 16, 16, 32)        320       
                                                                 
 conv2d_22 (Conv2D)          (None, 8, 8, 64)          18496     
                                                                 
 conv2d_23 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 flatten_4 (Flatten)         (None, 2048)              0         
                                                                 
 dense_7 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 96770 (378.01 KB)
Trainable params: 96770 (37

In [52]:
# np.prod(shape_before_flatten), shape_before_flatten

In [53]:
# 디코더
decorder_input = layers.Input(shape=(2,))
x = layers.Dense(np.prod(shape_before_flatten))(decorder_input) # 2048
x = layers.Reshape(shape_before_flatten)(x)
x = layers.Conv2DTranspose(128,(3,3),strides=2,activation='relu',padding='same')(x)
x = layers.Conv2DTranspose(64,(3,3),strides=2,activation='relu',padding='same')(x)
x = layers.Conv2DTranspose(32,(3,3),strides=2,activation='relu',padding='same')(x)
decoder_output = layers.Conv2D(1,(3,3),strides=1,activation='sigmoid',padding='same')(x) # 이미지 그려야..
decorder = models.Model(decorder_input,decoder_output)

In [54]:
decorder.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_8 (Dense)             (None, 2048)              6144      
                                                                 
 reshape_3 (Reshape)         (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose_3 (Conv2D  (None, 8, 8, 128)         147584    
 Transpose)                                                      
                                                                 
 conv2d_transpose_4 (Conv2D  (None, 16, 16, 64)        73792     
 Transpose)                                                      
                                                                 
 conv2d_transpose_5 (Conv2D  (None, 32, 32, 32)        184

In [55]:
#오토인코더 : 인코더+디코더
autoencorder = models.Model(encorder_input, decorder(encorder_output))  # 이미지를 입력으로 받아서 인코더와 디코더를 통과

In [56]:
autoencorder.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_21 (Conv2D)          (None, 16, 16, 32)        320       
                                                                 
 conv2d_22 (Conv2D)          (None, 8, 8, 64)          18496     
                                                                 
 conv2d_23 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 flatten_4 (Flatten)         (None, 2048)              0         
                                                                 
 dense_7 (Dense)             (None, 2)                 4098      
                                                                 
 model_10 (Functional)       (None, 32, 32, 1)         246

In [57]:
autoencorder.compile(optimizer='adam', loss='binary_crossentropy')

In [58]:
x_train.shape, y_train.shape

((60000, 32, 32, 1), (60000,))

In [ ]:
autoencorder.fit(x_train,x_train,epochs=5,batch_size=100,
                 shuffle=True,validation_data=(x_test,y_test))

Epoch 1/5
 21/600 [>.............................] - ETA: 2:46 - loss: 0.5977